In [ ]:
!nvidia-smi

Fri Jul  9 08:17:30 2021       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 470.42.01    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   62C    P8    10W /  70W |      0MiB / 15109MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [ ]:
!cp /content/drive/MyDrive/colab_data/data/CASIA-maxpy-clean.xv ./

In [ ]:
! tar xfJ CASIA-maxpy-clean.xv

In [ ]:
import os, shutil

dirs = os.listdir('CASIA-maxpy-clean')
print(len(dirs))

for d in dirs:
  path = os.path.join('CASIA-maxpy-clean', d)
  if len(os.listdir(path))>=20:
    continue
  shutil.rmtree(path)

dirs = os.listdir('CASIA-maxpy-clean')
print(len(dirs))

1120
640


In [ ]:
import torch
print(torch.__version__)
print(torch.version.cuda)

1.9.0+cu102
10.2


In [ ]:
import sys
sys.path.append('/content/drive/MyDrive/colab_data/packages/repmlp')
#sys.path=['', '/env/python', '/usr/lib/python36.zip', '/usr/lib/python3.6', '/usr/lib/python3.6/lib-dynload', '/usr/local/lib/python3.6/dist-packages', '/usr/lib/python3/dist-packages', '/usr/local/lib/python3.6/dist-packages/IPython/extensions', '/root/.ipython']
print(sys.path)

['', '/content', '/env/python', '/usr/lib/python37.zip', '/usr/lib/python3.7', '/usr/lib/python3.7/lib-dynload', '/usr/local/lib/python3.7/dist-packages', '/usr/lib/python3/dist-packages', '/usr/local/lib/python3.7/dist-packages/IPython/extensions', '/root/.ipython', '/content/drive/MyDrive/colab_data/packages/repmlp']


In [ ]:
def load_checkpoint(model, ckpt_path):
    checkpoint = torch.load(ckpt_path)
    if 'state_dict' in checkpoint:
        checkpoint = checkpoint['state_dict']
    ckpt = {}
    for k, v in checkpoint.items():
        if k.startswith('module.'):
            ckpt[k[7:]] = v
        else:
            ckpt[k] = v
    model.load_state_dict(ckpt)

In [ ]:
import glob
import os
import random

import numpy as np

import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

from PIL import Image
from sklearn.model_selection import train_test_split
from torch.optim.lr_scheduler import StepLR
from torch.utils.data import DataLoader, Dataset
from torchvision import datasets, transforms
from tqdm.notebook import tqdm

from repmlp.repmlp_resnet import *
#from utils import accuracy, ProgressMeter, AverageMeter, load_checkpoint

# checkpoint
PRETRAINED = '/content/drive/MyDrive/colab_data/models/RepMLP-Res50-light-224_train.pth'
CHECKPOINT = '/content/drive/MyDrive/colab_data/outputs/RepMLP/checkpoint.pt'
total_epochs = 0

# 训练图片路径
train_dir = '/content/CASIA-maxpy-clean'


# Training settings
batch_size = 32
epochs = 20
lr = 3e-5
gamma = 0.7
seed = 42

print(f"batch_size={batch_size}, epochs={epochs}, lr={lr}")


def seed_everything(seed):
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)
    torch.backends.cudnn.deterministic = True

seed_everything(seed)


#device = 'cpu'
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

print(f"Device: {device}")

os.makedirs('data', exist_ok=True)

# 准备 labels
label_list = os.listdir(train_dir)
label_dict = { v:i for i,v in enumerate(label_list)}
print(f"num_classes: {len(label_list)}")

# load data
train_list = glob.glob(os.path.join(train_dir+'/*','*.jpg'))
print(f"Train Data: {len(train_list)}")

labels = [path.split('/')[-2] for path in train_list]

# split
train_list, valid_list = train_test_split(train_list, 
    test_size=0.2, 
    stratify=labels, # 对有些数据集需要注释掉，标签下只有1个数据的情况
    random_state=seed)
print(f"Train Set: {len(train_list)}")
print(f"Validation Set: {len(valid_list)}")


# Image Augumentation
train_transforms = transforms.Compose(
    [
        transforms.Resize((224, 224)),
        transforms.RandomResizedCrop(224),
        transforms.RandomHorizontalFlip(),
        transforms.ToTensor(),
    ]
)

val_transforms = transforms.Compose(
    [
        transforms.Resize((224, 224)),
        transforms.RandomResizedCrop(224),
        transforms.RandomHorizontalFlip(),
        transforms.ToTensor(),
    ]
)


test_transforms = transforms.Compose(
    [
        transforms.Resize((224, 224)),
        transforms.RandomResizedCrop(224),
        transforms.RandomHorizontalFlip(),
        transforms.ToTensor(),
    ]
)


# Load Datasets

class FaceDataset(Dataset):
    def __init__(self, file_list, transform=None):
        self.file_list = file_list
        self.transform = transform

    def __len__(self):
        self.filelength = len(self.file_list)
        return self.filelength

    def __getitem__(self, idx):
        img_path = self.file_list[idx]
        img = Image.open(img_path)
        img_transformed = self.transform(img)

        label = img_path.split("/")[-2]

        return img_transformed, label_dict[label]

train_data = FaceDataset(train_list, transform=train_transforms)
valid_data = FaceDataset(valid_list, transform=test_transforms)

train_loader = DataLoader(dataset = train_data, batch_size=batch_size, shuffle=True )
valid_loader = DataLoader(dataset = valid_data, batch_size=batch_size, shuffle=True)


# model
model = create_RepMLPRes50_Light_224(deploy=False)

parameters = filter(lambda p: p.requires_grad, model.parameters())
parameters = sum([np.prod(p.size()) for p in parameters]) / 1_000_000
print('Trainable Parameters: %.3fM' % parameters)

if not torch.cuda.is_available():
    print('using CPU, this will be slow')
    use_gpu = False
else:
    model = model.cuda()
    use_gpu = True

# Training


# loss function
criterion = nn.CrossEntropyLoss()
# optimizer
optimizer = optim.Adam(model.parameters(), lr=lr)
# scheduler
scheduler = StepLR(optimizer, step_size=1, gamma=gamma)


if os.path.exists(CHECKPOINT):
    checkpoint = torch.load(CHECKPOINT)
    model.load_state_dict(checkpoint['model_state_dict'])
    optimizer.load_state_dict(checkpoint['optimizer_state_dict'])
    total_epochs = checkpoint['epoch']
    last_loss = checkpoint['loss']
    last_label = checkpoint['label_dict']
    print(f"Loaded {CHECKPOINT}: epochs= {total_epochs}, loss= {last_loss:.6f}, num_classes= {len(last_label)}")
else:
    # 载入 pre-train checkpoint
    if os.path.isfile(PRETRAINED):
        print("=> loading pre-trained checkpoint '{}'".format(PRETRAINED))
        load_checkpoint(model, PRETRAINED)


for epoch in range(epochs):
    epoch_loss = 0
    epoch_accuracy = 0

    for data, label in tqdm(train_loader):
        data = data.to(device)
        label = label.to(device)

        output = model(data)
        loss = criterion(output, label)

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        acc = (output.argmax(dim=1) == label).float().mean()
        epoch_accuracy += acc / len(train_loader)
        epoch_loss += loss / len(train_loader)

    with torch.no_grad():
        epoch_val_accuracy = 0
        epoch_val_loss = 0
        for data, label in valid_loader:
            data = data.to(device)
            label = label.to(device)

            val_output = model(data)
            val_loss = criterion(val_output, label)

            acc = (val_output.argmax(dim=1) == label).float().mean()
            epoch_val_accuracy += acc / len(valid_loader)
            epoch_val_loss += val_loss / len(valid_loader)

    print(
        f"Epoch : {epoch+1} - loss : {epoch_loss:.4f} - acc: {epoch_accuracy:.4f} - val_loss : {epoch_val_loss:.4f} - val_acc: {epoch_val_accuracy:.4f}\n"
    )

# 保存
torch.save({
            'epoch'                : total_epochs+epochs,
            'model_state_dict'     : model.state_dict(),
            'optimizer_state_dict' : optimizer.state_dict(),
            'loss'                 : epoch_loss,
            'label_dict'           : label_dict,
            }, CHECKPOINT)



batch_size=32, epochs=20, lr=3e-05
Device: cuda:0
num_classes: 640
Train Data: 22779
Train Set: 18223
Validation Set: 4556
Trainable Parameters: 87.038M
=> loading pre-trained checkpoint '/content/drive/MyDrive/colab_data/models/RepMLP-Res50-light-224_train.pth'


/usr/local/lib/python3.7/dist-packages/torch/nn/functional.py:718: UserWarning: Named tensors and all their associated APIs are an experimental feature and subject to change. Please do not use them for anything important until they are released as stable. (Triggered internally at  /pytorch/c10/core/TensorImpl.h:1156.)
  return torch.max_pool2d(input, kernel_size, stride, padding, dilation, ceil_mode)



Epoch : 1 - loss : 6.5202 - acc: 0.0167 - val_loss : 6.0316 - val_acc: 0.0478




Epoch : 2 - loss : 5.4783 - acc: 0.0997 - val_loss : 5.0759 - val_acc: 0.1380




Epoch : 3 - loss : 4.4499 - acc: 0.2071 - val_loss : 4.2649 - val_acc: 0.2314




Epoch : 4 - loss : 3.6624 - acc: 0.3133 - val_loss : 3.7280 - val_acc: 0.3006



In [ ]:
model = None
torch.cuda.empty_cache()

import gc
gc.collect()